In [3]:
import os, json

In [4]:
def perfect_match(v1,v2,k):
    if(len(v1)!=len(v2)):
        return 0
    delta = abs(v1[0] - v2[0])
    for i in range(len(v1)):
        if(abs(v1[i]-v2[i])-delta > k):
            #print(delta, v1[i],v2[i])
            return 0
    return 1

def perfect_align_clustering(phrases_vec,k):
    mp = {}
    remap = {}
    id = 0
    phrases = []
    for phrase, vec in phrases_vec.items():
        if(len(vec) > 1):#phrase appearing only one time is trievally perfectly align with any other one-time phrase and discard them 
            phrases.append(phrase)

    for i in range(len(phrases)):
        pi = phrases[i]
        vi = phrases_vec[pi]
        if(pi not in mp):
            mp[pi] = id
            remap[id] = [pi]
            id += 1
        else:
            continue
        for j in range(i+1, len(phrases)):
            pj = phrases[j]
            vj = phrases_vec[pj]

            #optimization: pj in mp denotes that pj must be at least perfectly match with one phrase before pi, but pi is not in mp
            #so pi and pj must be not perfectly aligned, and thus skip comparison 
            if(pj in mp):
                continue

            if(perfect_match(vi,vj,k) == 1):
                mp[pj] = id-1
                remap[id-1].append(pj)
    
    return mp, remap

In [5]:


def read_dict(file):
    print(file)
    with open(file, 'r') as file:
        data = json.load(file)
    return data 

def read_file(file):
    data = []
    with open(file, 'r') as file:
        # Iterate over each line in the file
        for line in file:
            # Print the line (you can replace this with other processing logic)
            data.append(line.strip())
    return data

def get_extracted_path(path):
    path = path.replace('raw','extracted')
    path = path.replace('.pdf','.txt')
    return path

def get_relative_location_path(extracted_path):
    path = extracted_path[:-4] + '_relative_location.csv'
    return path

def get_root_path():
    current_path = os.path.abspath(os.path.dirname(__file__))
    parent_path = os.path.abspath(os.path.join(current_path, os.pardir))
    #print("Parent path:", parent_path)
    return parent_path

def get_key_path(extracted_path):
    path = extracted_path.replace('extracted','truths/key_truth')
    return path

In [25]:
root_path = '/Users/yiminglin/Documents/Codebase/Pdf_reverse'
tested_paths = []
tested_paths.append(root_path + '/data/raw/complaints & use of force/Champaign IL Police Complaints/Investigations_Redacted.pdf')
tested_paths.append(root_path + '/data/raw/complaints & use of force/UIUC PD Use of Force/22-274.releasable.pdf')
tested_paths.append(root_path + '/data/raw/certification/CT/DecertifiedOfficersRev_9622 Emilie Munson.pdf')
tested_paths.append(root_path + '/data/raw/certification/IA/Active_Employment.pdf')
tested_paths.append(root_path + '/data/raw/certification/MT/RptEmpRstrDetail Active.pdf')
tested_paths.append(root_path + '/data/raw/certification/VT/Invisible Institue Report.pdf')

tested_id = 2 #starting from 0

path = tested_paths[tested_id]
extracted_path = get_extracted_path(path)
out_path = get_relative_location_path(extracted_path)
phrases = read_dict(out_path)
mp, remap = perfect_align_clustering(phrases,1)



/Users/yiminglin/Documents/Codebase/Pdf_reverse/data/extracted/certification/CT/DecertifiedOfficersRev_9622 Emilie Munson_relative_location.csv


In [35]:
print(phrases['11/18/2021'])
print(phrases['Bridgeport Police Department'])
print(phrases['Bridgeport Police Department'])


[40, 45]
[43, 48]


In [33]:
import key 
import importlib

importlib.reload(key)
s1 = [8, 112, 236, 351, 451, 552]
s2 = [4, 108, 232, 347, 447, 548]

key.perfect_match(s2,s1,1)

0
0
0
0
0
0


1

In [58]:
import sys
sys.path.append('/Users/yiminglin/Documents/Codebase/Pdf_reverse/')

from model import model 
model_name = 'gpt4o'

def result_gen_from_response(response):
    if('|' not in response and 'no' in response.lower()):
        return 0
    l = response.split('|')
    return len(l)+1

def cluster_filtering(clusters):
    #trick 1: drop singleton cluster 
    #for each remaining cluster, if half of phrases are not header, drop this cluster 
    instruction = 'The following list contains possibly keys and values extracted from a table. Return to me all the keys without explanation, and seperate each key by |. If no key is found, return NO.' 
    mp = {}
    for cid, l in clusters.items():
        context = ", ".join(l)
        prompt = (instruction,context)
        response = model(model_name,prompt)
        key_num = result_gen_from_response(response)
        if(key_num >= len(l)/2):
            mp[cid] = l

    return mp #mp is a new cluster after filtering by using LLM 

mp = cluster_filtering(remap)
for k,v in mp.items():
    print(k,v)

0 ['Officer Detail Reports #A-12', 'University of Illinois Police Department', 'Case Number', 'Date', 'Time', 'Assist', 'Agency', 'Information Taken From', 'Crime/ Incident', 'Status', 'Completed', 'Incident Reviewed By', 'Date Reviewed', 'Drugs/Alcohol', 'Location', 'K-9', 'Crisis Intervention Related', 'Type Premises', 'District', 'Recorded On Camera', 'Type Situation', 'Reason Force Used', 'Officer Injured', 'Danger Factors', 'Final Disposition', 'Verbal De-Escalation Attempted', '1']
1 ['Page 1 of 27', 'Page 2 of 27', '3/19/2021', 'Physical Resistance', 'EH Control - Physical Restraint', 'Osterholt, Drew L.', 'Jones, Kent R.', 'Page 6 of 27', '3/29/2021', 'Page 7 of 27']
4 ['SCTF', 'Snow, Ryan S.', 'incident.', 'Firearm', 'UPD', 'Wooded Area', 'Arrested/Lodged', 'Unknown', 'Germany, Karl', 'Page 4 of 27', 'Released / No Action', 'Page 5 of 27', 'Shooting Suspect']
10 ['Body Camera']
11 ['Fled Scene', 'Fleeing', 'Page 3 of 27']
13 ['Notes']
14 ['Subject #', 'DOB', 'Gender', 'Male', 

In [ ]:
def clustering_group(clusters):
    c = {}
    for cid, l in clusters.items():
        c[cid] = len(l)
    sc = dict(sorted(c.items(), key=lambda item: item[1]))
    l = []
    for k,v in sc.items():
        l.append(v)
    return l
    

l=clustering_group(remap)
print(l)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 6, 10, 13, 19, 27]


## Observation

 - totally relying on GPT to filter out clusters may introduce false negative, since one phrase can be both key and value semantically if no context is provided 

In [ ]:
print(phrases['Officer'])
print(phrases['Case Number'])

[90, 204, 318, 334, 350, 366, 478, 494, 510, 655, 671, 687, 703, 814, 924]
[6, 120, 232, 397, 543, 733, 843]


In [ ]:
s1 = [6, 120, 232, 397, 543, 733, 843]
s2 = [18, 132, 244, 409, 554, 745, 855]

perfect_match(s1,s2)

12 543 554


0

## Observation 
- Perfect aligned is too strict. Relax it to be the delta distance. 

## Observation
- Phrases with a short vector is more likely to be perfectly aligned with the phrase with larger vector, introducing FP.

Example 1: 

CPD [129, 406]

Action Taken [92, 206, 320, 336, 352, 368, 480, 496, 512, 657, 673, 689, 705, 816, 926]

Example 2:

Fled Scene [44, 270]

DOB [58, 172, 284, 448, 593, 625, 784, 894]